In [1]:
include("DiGraph.jl")
using .DiGraphs

In [2]:
expr = :(a = b + c)

:(a = b + c)

In [3]:
expr.args

2-element Array{Any,1}:
 :a      
 :(b + c)

In [4]:
s = expr.head

:(=)

In [5]:
struct Node
    varname::Symbol
    op::Symbol
end

In [7]:
xor_edges = [[Node(:x1, :input) Node(:z1, :!)];
    [Node(:x2, :input) Node(:z2, :!)];
    [Node(:x1, :input) Node(:z3, :&)];
    [Node(:x2, :input) Node(:z4, :&)];
    [Node(:z1, :!) Node(:z4, :&)];
    [Node(:z2, :!) Node(:z3, :&)]
    [Node(:z3, :&) Node(:z5, :|)];
    [Node(:z4, :&) Node(:z5, :|)];
    [Node(:z5, :|) Node(:z6, :output)]
    ]

9×2 Array{Node,2}:
 Node(:x1, :input)  Node(:z1, :!)     
 Node(:x2, :input)  Node(:z2, :!)     
 Node(:x1, :input)  Node(:z3, :&)     
 Node(:x2, :input)  Node(:z4, :&)     
 Node(:z1, :!)      Node(:z4, :&)     
 Node(:z2, :!)      Node(:z3, :&)     
 Node(:z3, :&)      Node(:z5, :|)     
 Node(:z4, :&)      Node(:z5, :|)     
 Node(:z5, :|)      Node(:z6, :output)

In [8]:
xor_dag = DiGraph(xor_edges)

DiGraph{Node}(Set(Node[Node(:z1, :!), Node(:x2, :input), Node(:z2, :!), Node(:z3, :&), Node(:x1, :input), Node(:z4, :&), Node(:z5, :|), Node(:z6, :output)]), Node[Node(:x1, :input) Node(:z1, :!); Node(:x2, :input) Node(:z2, :!); … ; Node(:z4, :&) Node(:z5, :|); Node(:z5, :|) Node(:z6, :output)])

In [9]:
virtual_v = Node(:EMPTY, :EMPTY)

Node(:EMPTY, :EMPTY)

In [10]:
xor_edges .== xor_edges

9×2 BitArray{2}:
 1  1
 1  1
 1  1
 1  1
 1  1
 1  1
 1  1
 1  1
 1  1

In [11]:
inputs = [node for node in xor_dag.vertices if node.op == :input]

2-element Array{Node,1}:
 Node(:x2, :input)
 Node(:x1, :input)

In [21]:
sorted_vertices = topological_sort(xor_dag)

8-element Array{Node,1}:
 Node(:x1, :input) 
 Node(:z1, :!)     
 Node(:x2, :input) 
 Node(:z4, :&)     
 Node(:z2, :!)     
 Node(:z3, :&)     
 Node(:z5, :|)     
 Node(:z6, :output)

In [142]:
function opvertex_to_expr(v, dg)

    result = Expr(:(=))
    
    rhs = Expr(:call)
    rhs.args = vcat([v.op], [p.varname for p in in_neighbors(dg, v)])
    
    result.args = vcat([v.varname], rhs)
    
    return result
end


function retvertex_to_expr(v, dg)
    
    result = Expr(:return)
    result.args = [p.varname for p in in_neighbors(dg, v)]
    return result
end


function invertices_to_sig(invertices, dg, funcname::String)

    result = Expr(:call)
    result.args = vcat(Symbol(funcname), [v.varname for v in invertices])
    return result
    
end


invertices_to_sig (generic function with 1 method)

In [143]:
function dag_to_func(dg, funcname::String="myfunction")

    invertices = [v for v in dg.vertices if v.op == :input]
    
    # There must be at least one return vertex:
    retvertex = [v for v in dg.vertices if v.op == :output][1]

    opvertices = [v for v in topological_sort(dg) if (! in(v, invertices)) & (! (v == retvertex))]
    
    signature = invertices_to_sig(invertices, dg, funcname)
    operations = [opvertex_to_expr(v, dg) for v in opvertices]
    retline = [retvertex_to_expr(retvertex, dg)]
                                        
    body = Expr(:block)
    body.args = vcat(operations, retline)
    
    result = Expr(:function)
    result.args = vcat([signature], [body])
                                        
    return result
                                        
end

dag_to_func (generic function with 2 methods)

In [144]:
dag_to_func(xor_dag)

:(function myfunction(x2, x1)
      z1 = !x1
      z4 = z1 & x2
      z2 = !x2
      z3 = z2 & x1
      z5 = z3 | z4
      return z5
  end)

In [145]:
eval(dag_to_func(xor_dag))

myfunction (generic function with 1 method)

In [147]:
for i = (true, false)
    for j = (true, false)
        println(i, "\t", j, "\t", myfunction(i,j))
    end
end

true	true	false
true	false	true
false	true	true
false	false	false
